In [1]:
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [2]:
from langchain_community.utilities import SerpAPIWrapper

# 구글 검색엔진, 검색 결과의 지역을 한국,언어를 한국어, 검색 결과를 3개만 반환
params = {"engine": "google", "gl": "kr", "hl": "ko", "num": "3"}

# 1. SerpAPIWrapper 인스턴스 생성
search = SerpAPIWrapper(params=params)

In [20]:
# 2. run 메서드를 사용하여 검색 실행
# 검색하고 싶은 키워드를 전달합니다.
query = "5호선 상일동역 주변에 있는 맛집"
search_result = search.run(query=query)

In [21]:
search_results = eval(search_result)
search_results

['1. 벨파아제 · Good Italian bistro ; 2. 강동반상 · good atmosphere, cost-effective Korean restaurant ; 3. 스키마야 · 라멘, 덮밥 추천 · 덮밥류가 맛있는 집 ; 4. 신불 ...',
 '서울특별시 강동구 상일로5길 8-7 1층. 저장. 전화. 버거비 상일동점. 서울특별시 강동구 천호대로221길 6 1층. 저장. 전화. 오한수우육면가 상일점. 서울 ...',
 [{'position': 1,
   'rating': 4.8,
   'reviews': 103,
   'reviews_original': '(103)',
   'price': '₩20,000~30,000',
   'description': '"평일에도 웨이팅 있는 맛집이니 정말 맛집으로 인정합니다."',
   'lsig': 'AB86z5UFl0mK4kmmAXrfFYIPVOk8',
   'thumbnail': 'https://serpapi.com/searches/68a58ba36561a65707bbbb8d/images/1d724752a7fe6b9c7d07a9c369de006552804776a7e39b96c6107ea3e995a1313c11c1859202628e.jpeg',
   'place_id': '12730341462943213745',
   'place_id_search': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=kr&google_domain=google.com&hl=ko&ludocid=12730341462943213745&num=3&q=5%ED%98%B8%EC%84%A0+%EC%83%81%EC%9D%BC%EB%8F%99%EC%97%AD+%EC%A3%BC%EB%B3%80%EC%97%90+%EC%9E%88%EB%8A%94+%EB%A7%9B%EC%A7%91',
   'gps_coordinates': {'latitude': 37.557636, 'longitud

In [26]:
# Gemini 2.5 Flash 모델을 사용하여 식당명을 추출합니다.
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List
import re
import ast


# 식당 정보를 담는 모델 정의
class Restaurant(BaseModel):
    name: str = Field(description="식당명")


# 여러 식당 정보를 담는 모델 정의
class RestaurantList(BaseModel):
    restaurants: List[Restaurant]


# OutputParser 초기화
parser = PydanticOutputParser(pydantic_object=RestaurantList)

# Gemini 2.5 Flash 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)

# 프롬프트 템플릿 생성
prompt_template = """
다음 검색 결과에서 식당명만을 추출해주세요.
각 검색 결과를 분석하여 식당 이름을 정확히 식별하고 JSON 형식으로 출력해주세요.

{format_instructions}

검색 결과:
{search_data}
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["search_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Gemini 모델을 사용하여 식당명 추출
chain = prompt | model | parser

# search_results를 문자열로 변환하여 모델에 전달
search_data_str = str(search_results)
result_obj = chain.invoke({"search_data": search_data_str})

# 파서로 JSON 출력
import json

json_str = result_obj.model_dump_json(indent=2)
print("=== JSON 출력 ===")
print(json_str)

print("\n=== 식당명 목록 ===")
# 보기 좋게 출력
for i, r in enumerate(result_obj.restaurants, 1):
    print(f"{i}. 식당명: {r.name}")
    print("-" * 30)

=== JSON 출력 ===
{
  "restaurants": [
    {
      "name": "벨파아제"
    },
    {
      "name": "강동반상"
    },
    {
      "name": "스키마야"
    },
    {
      "name": "버거비 상일동점"
    },
    {
      "name": "오한수우육면가 상일점"
    },
    {
      "name": "고반식당 상일동역점"
    },
    {
      "name": "송하정스시"
    },
    {
      "name": "쇠나무그릴"
    }
  ]
}

=== 식당명 목록 ===
1. 식당명: 벨파아제
------------------------------
2. 식당명: 강동반상
------------------------------
3. 식당명: 스키마야
------------------------------
4. 식당명: 버거비 상일동점
------------------------------
5. 식당명: 오한수우육면가 상일점
------------------------------
6. 식당명: 고반식당 상일동역점
------------------------------
7. 식당명: 송하정스시
------------------------------
8. 식당명: 쇠나무그릴
------------------------------
